In [1]:
import torch 
from torch import nn
from torch.nn import functional as F 

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

In [2]:
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [3]:
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1302,  0.1064,  0.1473,  0.1568,  0.0546, -0.1855,  0.0022, -0.1573,
          0.2028,  0.2338],
        [ 0.1433,  0.1804,  0.1138,  0.1144,  0.0348, -0.1467, -0.0069, -0.1453,
          0.2522,  0.1879]], grad_fn=<AddmmBackward0>)

In [4]:
# 自定义层和块

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.0273, -0.2053, -0.1904, -0.1838, -0.0585,  0.1427,  0.0113,  0.0141,
         -0.0358, -0.2853],
        [-0.0306,  0.0533, -0.1710, -0.0127, -0.0526,  0.2376,  0.0480, -0.0422,
         -0.0649, -0.1829]], grad_fn=<AddmmBackward0>)

In [6]:

# 顺序块 
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
            
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X  

记一下上面语法层面的内容：
- *args是可以多参数，会被封装成一个tuple，如果要解包成单个就再 用一个✳️
- _modules这里是一个dict python高版本的默认是ordered dict
- emirate是为了让我们可以获取索引

In [7]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(X)

tensor([[ 0.2618, -0.1365,  0.1026, -0.3732,  0.3551,  0.0354, -0.0680,  0.0471,
          0.0826, -0.0928],
        [ 0.3009, -0.1688,  0.1154, -0.1049,  0.2185,  0.0253, -0.0621, -0.0304,
          0.1206, -0.2205]], grad_fn=<AddmmBackward0>)

In [10]:
# 下面这个例子目前看起来没有大用，只是为了演示灵活性，我们可以更多自定义 
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rando_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rando_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [11]:
net = FixedHiddenMLP()
net(X)

tensor(0.0572, grad_fn=<SumBackward0>)

In [12]:
# 还可以做各种混合 
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(X)

tensor(0.0416, grad_fn=<SumBackward0>)